<h2>Задание 1.1</h2>

In [29]:
import pandas as pd
file_path = 'shopping_trends.csv'

try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path}' не найден. Проверьте путь и повторите попытку.")
    exit()

df=data.copy()
subscribed = df[df['Subscription Status'] == 'Yes']
not_subscribed = df[df['Subscription Status'] == 'No']
express_subscribed = subscribed[subscribed['Shipping Type'] == 'Express']
express_not_subscribed = not_subscribed[not_subscribed['Shipping Type'] == 'Express']

print(len(express_subscribed) / len(subscribed))
print(len(express_not_subscribed) / len(not_subscribed))

0.18043684710351376
0.1601685985247629


Доля покупателей с подпиской, использующих тип доставки "Express": 18.04%
Доля покупателей без подписки, использующих тип доставки "Express": 16.02%
Вывод:
Покупатели с подпиской действительно чуть чаще выбирают "Express" как тип доставки по сравнению с покупателями без подписки. Однако разница между этими группами составляет примерно 2%, что может быть статистически незначимой. Для более точных выводов рекомендуется провести тест на статистическую значимость (например, с помощью теста пропорций).

In [17]:
correlation = df['Previous Purchases'].corr(df['Purchase Amount (USD)'])
print(correlation)

0.008063412270587713



На основании вычисленного коэффициента корреляции 0.0081 можно сделать следующий вывод:
Связь между количеством предыдущих покупок и суммой текущей покупки практически отсутствует, так как значение корреляции близко к нулю. Это означает, что количество предыдущих покупок не является значимым предиктором суммы текущей покупки.

In [10]:
# Покупатели с высоким рейтингом
high_rated = df[df['Review Rating'] >= 4]
# Покупки с применением скидки
discounted_high_rated = high_rated[high_rated['Discount Applied'] == 'Yes']
print(len(discounted_high_rated) / len(high_rated))


0.42700729927007297



На основании вычисленного коэффициента корреляции можно сделать следующий вывод:
Существует умеренная положительная корреляция между рейтингом обзоров и использованием скидок: коэффициент 0.427 указывает на то, что покупатели с более высокими рейтингами обзоров (4-5) действительно имеют тенденцию чаще покупать товары с большими скидками. Это может свидетельствовать о том, что более удовлетворенные покупатели (с высоким рейтингом обзора) склонны искать и использовать скидки при совершении покупок.


In [21]:
free_shipping = df[df['Shipping Type'] == 'Free Shipping']
clothing_free_shipping = free_shipping[free_shipping['Category'] == 'Clothing']
print(len(clothing_free_shipping) / len(free_shipping))


0.43555555555555553


Существует умеренная положительная корреляция между выбором бесплатным типом доставки и покупками в категории "Clothing". Коэффициент 0.4356 указывает на то, что покупатели, заинтересованные в бесплатной доставке, могут быть более склонны выбирать товары из категории "Clothing", возможно, из-за более выгодных предложений или привычки покупок в этой категории.

In [27]:
california_purchases = df[df['Location'] == 'California']
outerwear_california = california_purchases[california_purchases['Category'] == 'Outerwear']
print(len(outerwear_california) / len(california_purchases))


0.06315789473684211



На основании вычисленного коэффициента корреляции 0.0632 можно сделать следующий вывод:
Связь между покупателями из штата "California" и покупками в категории "Outerwear" практически отсутствует, так как значение корреляции очень низкое и близко к нулю. Это говорит о том, что штат покупателя ("California") не является значимым фактором, определяющим склонность к покупкам в категории "Outerwear".

In [33]:
from scipy import stats
from collections import Counter

<h2>Задание 1.2</h2>

In [72]:
popular = df['Item Purchased'].value_counts().idxmax()
print("Самый популярный товар:", popular)

Самый популярный товар: Blouse


Распределение покупателей по полу

In [43]:
df['Gender'].value_counts()

Gender
Male      2652
Female    1248
Name: count, dtype: int64

In [52]:
total_purchase_gender = df.groupby('Gender')['Purchase Amount (USD)'].sum()
total_purchase_age = df.groupby('Age')['Purchase Amount (USD)'].sum()

frequency_gender = df['Gender'].value_counts()
frequency_age = df['Age'].value_counts()

most_expensive_gender = df.groupby('Gender')['Purchase Amount (USD)'].max()
most_expensive_age = df.groupby('Age')['Purchase Amount (USD)'].max()
print("Кто покупает больше всего по полу:\n", total_purchase_gender)
print("Кто покупает чаще всего по полу:\n", frequency_gender)
print("Самая дорогая покупка по полу:\n", most_expensive_gender)

Кто покупает больше всего по полу:
 Gender
Female     75191
Male      157890
Name: Purchase Amount (USD), dtype: int64
Кто покупает чаще всего по полу:
 Gender
Male      2652
Female    1248
Name: count, dtype: int64
Самая дорогая покупка по полу:
 Gender
Female    100
Male      100
Name: Purchase Amount (USD), dtype: int64


In [65]:
color_season_table = pd.crosstab(df['Color'], df['Season'])
# Хи-квадрат тест
chi2, p_value, dof, expected = stats.chi2_contingency(color_season_table)
if p_value < 0.05:
    print("Есть статистически значимая зависимость между цветом одежды и сезоном.")
else:
    print("Нет статистически значимой зависимости между цветом одежды и сезоном.")

Нет статистически значимой зависимости между цветом одежды и сезоном.


In [67]:
df['Season'] = df['Season'].str.lower()  # Приводим к единому виду
mau_season = df.groupby('Season')['Customer ID'].nunique()
mau_dynamic = mau_season.pct_change()
print("Сезонный MAU:\n", mau_season)
print("Динамика MAU:\n", mau_dynamic)

Сезонный MAU:
 Season
fall      975
spring    999
summer    955
winter    971
Name: Customer ID, dtype: int64
Динамика MAU:
 Season
fall           NaN
spring    0.024615
summer   -0.044044
winter    0.016754
Name: Customer ID, dtype: float64


In [68]:
clothing_items = df[df['Category'] == 'Clothing']['Item Purchased']
letters = ''.join(clothing_items).lower()
most_common_letter = Counter(letters).most_common(1)[0][0]
print("Самая популярная буква в названии одежды:", most_common_letter)

Самая популярная буква в названии одежды: s


<h1>Задание 2</h1>


In [78]:
file_path = 'drinks.csv'

try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path}' не найден. Проверьте путь и повторите попытку.")
    exit()

df=data.copy()
# Поиск стран с максимальным потреблением
max_beer = df.loc[df['beer_servings'].idxmax()]
max_spirit = df.loc[df['spirit_servings'].idxmax()]
max_wine = df.loc[df['wine_servings'].idxmax()]

print(f"Страна с наибольшим потреблением пива: {max_beer['country']} ({max_beer['beer_servings']} servings)")
print(f"Страна с наибольшим потреблением крепкого алкоголя: {max_spirit['country']} ({max_spirit['spirit_servings']} servings)")
print(f"Страна с наибольшим потреблением вина: {max_wine['country']} ({max_wine['wine_servings']} servings)")
print("\nСравнение стран с максимальным потреблением алкоголя:")
print(df.loc[[max_beer.name, max_spirit.name, max_wine.name]])


Страна с наибольшим потреблением пива: Namibia (376 servings)
Страна с наибольшим потреблением крепкого алкоголя: Grenada (438 servings)
Страна с наибольшим потреблением вина: France (370 servings)

Сравнение стран с максимальным потреблением алкоголя:
     country  beer_servings  spirit_servings  wine_servings  \
117  Namibia            376                3              1   
68   Grenada            199              438             28   
61    France            127              151            370   

     total_litres_of_pure_alcohol  
117                           6.8  
68                           11.9  
61                           11.8  


<h1>Задание 3</h1>

In [85]:
file_path = 'tarantino.csv'

try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path}' не найден. Проверьте путь и повторите попытку.")
    exit()

df=data.copy()
curse = df[df['type'] == 'word'].groupby('movie')['word'].count().reset_index(name='curse')
death = df[df['type'] == 'death'].groupby('movie')['type'].count().reset_index(name='death')
movie_stats = pd.merge(curse, death, on='movie', how='outer').fillna(0)
movie_stats['curse_to_death_ratio'] = movie_stats['curse'] / movie_stats['death']
word_frequency = df[df['type'] == 'word']['word'].value_counts().reset_index()
word_frequency.columns = ['word', 'frequency']
curse_times = df[df['type'] == 'word'].sort_values('minutes_in')['minutes_in']
time_differences = curse_times.diff().dropna().reset_index(drop=True)

print("Число проклятий, летальных исходов и их соотношение по фильмам:\n", movie_stats)
print("\nЧастота употребления конкретных проклятий:\n", word_frequency)
print("\nРаспределение времени между проклятиями:\n", time_differences)

Число проклятий, летальных исходов и их соотношение по фильмам:
                  movie  curse  death  curse_to_death_ratio
0     Django Unchained    262     47              5.574468
1  Inglorious Basterds     58     48              1.208333
2         Jackie Brown    368      4             92.000000
3    Kill Bill: Vol. 1     57     63              0.904762
4    Kill Bill: Vol. 2     69     11              6.272727
5         Pulp Fiction    469      7             67.000000
6       Reservoir Dogs    421     10             42.100000

Частота употребления конкретных проклятий:
              word  frequency
0         fucking        407
1            shit        221
2            fuck        213
3         n-word         179
4             ass        140
5         goddamn        113
6    motherfucker         70
7           bitch         64
8            hell         45
9            damn         37
10  motherfucking         27
11         fucked         25
12  motherfuckers         15
13       bul